# 5. 分支鏈 (Chains Branching) - Ollama 版本

本範例展示根據條件選擇不同處理路徑，如客戶回饋分類處理，使用 RunnableBranch 實現條件分支。

## 學習重點
- 使用 RunnableBranch 實現條件分支
- 學習如何根據輸入內容選擇不同的處理路徑
- 理解分支邏輯的設計模式
- 掌握複雜條件判斷的實現方法


In [ ]:
# 導入必要的套件
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch
from langchain_ollama.llms import OllamaLLM

# 載入環境變數
load_dotenv()

# 建立 Ollama 模型
model = OllamaLLM(model="llama3.2:latest")


In [ ]:
# 定義不同回饋類型的提示模板

# 正面回饋模板
positive_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個友善的助手。"),
        ("human", "為這個正面回饋生成感謝訊息：{feedback}。"),
    ]
)

# 負面回饋模板
negative_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個友善的助手。"),
        ("human", "為這個負面回饋生成回應：{feedback}。"),
    ]
)

# 中性回饋模板
neutral_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個友善的助手。"),
        ("human", "為這個中性回饋生成要求更多細節的訊息：{feedback}。"),
    ]
)

# 升級回饋模板
escalate_feedback_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個友善的助手。"),
        ("human", "為這個回饋生成升級給人工客服的訊息：{feedback}。"),
    ]
)

print("回饋處理模板已定義：")
print("1. positive_feedback_template: 正面回饋")
print("2. negative_feedback_template: 負面回饋")
print("3. neutral_feedback_template: 中性回饋")
print("4. escalate_feedback_template: 升級回饋")


In [ ]:
# 定義回饋分類模板
classification_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個友善的助手。"),
        ("human", "將這個回饋分類為正面、負面、中性或升級：{feedback}。"),
    ]
)

print("回饋分類模板已定義：classification_template")


In [ ]:
# 定義分支處理邏輯
# 使用 RunnableBranch 根據分類結果選擇不同的處理路徑
branches = RunnableBranch(
    # 正面回饋分支
    (
        lambda x: "正面" in x or "positive" in x.lower(),
        positive_feedback_template | model | StrOutputParser()
    ),
    # 負面回饋分支
    (
        lambda x: "負面" in x or "negative" in x.lower(),
        negative_feedback_template | model | StrOutputParser()
    ),
    # 中性回饋分支
    (
        lambda x: "中性" in x or "neutral" in x.lower(),
        neutral_feedback_template | model | StrOutputParser()
    ),
    # 預設分支（升級處理）
    escalate_feedback_template | model | StrOutputParser()
)

print("分支處理邏輯已定義：")
print("1. 正面回饋 → 感謝訊息")
print("2. 負面回饋 → 回應處理")
print("3. 中性回饋 → 要求更多細節")
print("4. 其他情況 → 升級給人工客服")


In [ ]:
# 建立分類鏈
classification_chain = classification_template | model | StrOutputParser()

# 組合分類和回應生成鏈
chain = classification_chain | branches

print("完整的分支鏈已建立！")
print("鏈的流程：回饋輸入 → 分類 → 選擇處理路徑 → 生成回應")


In [ ]:
# 測試不同類型的回饋
test_feedbacks = [
    "產品很棒！我非常滿意，推薦給大家！",  # 正面回饋
    "產品品質很差，完全不值這個價錢。",  # 負面回饋
    "產品還可以，沒有特別的感覺。",      # 中性回饋
    "我不確定這個產品是否適合我。"       # 不確定回饋
]

print("=" * 60)
print("回饋處理測試結果：")
print("=" * 60)

for i, feedback in enumerate(test_feedbacks, 1):
    print(f"\n測試 {i}: {feedback}")
    print("-" * 40)
    result = chain.invoke({"feedback": feedback})
    print(f"回應: {result}")
    print("=" * 60)


## 💡 重點說明

### 分支鏈的設計模式

1. **分類階段**: 首先對輸入進行分類
2. **條件判斷**: 根據分類結果選擇處理路徑
3. **分支處理**: 每個分支都有專門的處理邏輯
4. **預設處理**: 提供預設分支處理未分類的情況

### RunnableBranch 的使用

```python
branches = RunnableBranch(
    (條件函數1, 處理鏈1),
    (條件函數2, 處理鏈2),
    (條件函數3, 處理鏈3),
    預設處理鏈  # 最後一個是預設分支
)
```

### 條件函數的設計

```python
# 條件函數應該返回 True 或 False
lambda x: "正面" in x or "positive" in x.lower()
```

### 實際應用場景

- 客戶服務自動化
- 內容分類處理
- 智能路由系統
- 多條件決策系統
